In [40]:
# Import required modules
%load_ext autoreload
%autoreload 2

import numpy as np
import os.path
import pickle

import argparse
import torch

import random
import numpy.random
import torch.random

from datetime import datetime

import warnings
warnings.filterwarnings(action='ignore')

In [41]:
# Define variables
parser = argparse.ArgumentParser()
args, unknown = parser.parse_known_args()

# 2023-05-02
# For Using GPU (CUDA)
args.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 2023-05-02
# For using tensorboard
args.istensorboard = False

args.total_episode = 100
args.learning_rate = 0.1 # Learning rate Alpha
args.boltzmann_tau_initial = 5 # Initial temperature parameter at Boltzmann policy, Tau
args.total_reward = 0
args.epsilon = 0.1

args.max_iteration = 50 # Maximum iteration num. of algorithm, MAX_STEPS
args.replay_batch = 100 # Replay batch size, B
args.nn_update_num = 20 # CNN update number, U: [(1) Constant num. of iteration], (2) Lower limit of loss function value
args.batch_size = 20 # Batch size, N
args.replay_memory = [] # (1) Using list class, (2) Replay memory class by SLM Lab (page. 105~108)

args.gridnum_x = 15
args.gridnum_y = 15
args.gridsize_x = 120 # ft
args.gridsize_y = 120 # ft

args.time_step = 120 # days
args.total_production_time = 600 # days

# 2023-05-02: For reproduction
args.random_seed = 202022673
random.seed(args.random_seed)
np.random.seed(args.random_seed)
torch.manual_seed(args.random_seed)

# State: Pressure distribution, Oil saturation

# Action: Well placement (Coordinate of well location)

# Environment: Reservoir simulator

# Reward: NPV at each time segment

args.discount_rate = 0.1 # Used for calculation of NPV
args.discount_factor = 1 # Used for Q-value update

args.simulation_directory = "H:\\Lab_Meeting\\Simulation_Model\\2D_15-by-15_DQN\\2D_15-by-15_DQN_Filetest"

In [44]:
# with open("H:\\Lab_Meeting\\Simulation_Model\\2D_15-by-15_DQN\\2D_15-by-15_DQN_Filetest\\2D_JY_ECLRUN_1.PRT", 'r') as file_read:
#     line = file_read.readline()
#     print(line)
#     while not line.startswith(f"  PRESSURE AT   {120*1}"):
#         line = file_read.readline()
#     for i in range(1,10+1):
#         line = file_read.readline()
#     print(line)
#
#     line_temp = []
#     lines_converted = []
#     for i in range(1, 15+1):
#         # line_temp = [element.strip() for element in line.split()]
#         lines_converted.append([element.strip() for element in line.split()][3::])
#         line = file_read.readline()
#     print(lines_converted)
# print(f"  PRESSURE AT   {120*1}")

def _read_ecl_prt(args, idx: int, tstep_idx: int, filename: str, dynamic_type: str) -> list:
    # Check if dynamic type input is (1) 'PRESSURE' or (2) 'SOIL'
    if not (dynamic_type == 'PRESSURE' or 'SOIL'):
        print("Assign correct dynamic data input type!")
        exit()

    # File IO
    # 1. Open .PRT file
    with open(args.simulation_directory+'\\'+filename+'_'+str(idx)+'.PRT') as file_read:
        line = file_read.readline()
        # 2. Fine the location of dynamic data
        while not line.startswith(f"  {dynamic_type} AT   {args.time_step * tstep_idx}"):
            line = file_read.readline()
        # 3. Dynamic data is located at 10th line below the line ["  {dynamic_type} AT   {args.time_step * tstep_idx}"]
        for i in range(1,10+1):
            line = file_read.readline()
        # 4. Collect dynamic data
        lines_converted = []
        for i in range(1, args.gridnum_y+1):
            lines_converted.append([element.strip() for element in line.split()][3::])
            line = file_read.readline()

    return lines_converted

test = _read_ecl_prt(args, 1, 1, '2D_JY_ECLRUN', 'PRESSURE')

print(test)

[['2896.53', '2801.27', '2668.91', '2384.10', '1984*63', '2246.24', '2554.42', '2856.06', '2877.74', '2638.75', '2045.21', '1714.65', '1637.40', '1612.53', '1609.95'], ['2790.04', '2742.56', '2645.40', '2452.54', '2269.98', '2297.21', '2389.38', '2466.83', '2380.28', '2094.32', '1898.41', '1698.56', '1624.00', '1606.58', '1603.59'], ['2710.02', '2667.83', '2570.33', '2440.28', '2324.07', '2238.68', '2189.47', '2180.97', '2031.79', '1877.39', '1754.63', '1661.49', '1605.74', '1592.54', '1590.28'], ['2611.85', '2580.33', '2456.93', '2363.23', '2187.79', '2046.79', '1985.28', '1905.04', '1770.26', '1679.24', '1640.66', '1609.55', '1581.99', '1577.25', '1577.90'], ['2500.25', '2422.37', '2296.76', '2200.01', '1952.87', '1878.80', '1792.09', '1716.47', '1648.39', '1622.20', '1603.89', '1581.42', '1562.48', '1566.53', '1569.50'], ['2347.99', '2235.47', '2184.07', '1997.50', '1796.98', '1732.41', '1670.44', '1635.46', '1613.06', '1601.27', '1589.73', '1568.60', '1537*60', '1556.66', '1563.24'

In [ ]:
# def _get_griddata(self, idx, tstep_idx, filename, data_type):
#     nx, ny = self.args.num_of_x, self.args.num_of_y
#
#     with open(self.simulation_directory+'/'+filename+'_'+str(idx)+'.F'+'0'*(4-len(str(tstep_idx)))+str(tstep_idx)) as f:
#         lines = f.readlines()
#
#     lines_converted = []
#     for line in lines:
#         lines_converted.append([element.strip() for element in line.split()])
#
#     condition = False
#     data = []
#
#     for line in lines_converted:
#         if f"'{data_type}" in line or f"'{data_type}'" in line:
#             condition = True
#         elif condition:
#             data += [float(l) for l in line]
#         if len(data) == nx*ny:
#             break
#
#     return data